# Rxiv-Maker: Automated LaTeX Article Generation (Docker-Accelerated)

This notebook allows you to run Rxiv-Maker in Google Colab using Docker containers for **significantly faster setup** and more reliable execution.

**Features:**
- Convert Markdown to LaTeX with academic formatting
- Generate figures from Python scripts and Mermaid diagrams
- Complete LaTeX compilation with bibliography
- Download generated PDF and source files
- **⚡ Fast setup**: ~4 minutes instead of ~20 minutes
- **🔒 Reliable**: Pre-configured environment with all dependencies

**What you'll need:**
- Your manuscript in Markdown format with YAML metadata
- Bibliography file (.bib)
- Figure source files (Python scripts or Mermaid diagrams)

**Technical Note:** This notebook uses `udocker` to run Docker containers in Google Colab. The containers are optimized for AMD64 architecture but work on all platforms via emulation when needed.

## 🐳 Docker Setup and Installation

First, let's install udocker and pull the pre-configured Rxiv-Maker container with all dependencies.

In [ ]:
# @title Install udocker and pull Rxiv-Maker container
print("🐳 Setting up Docker environment...")
print("This takes ~4 minutes instead of ~20 minutes for manual dependency installation.")
print("=" * 70)

# Install udocker (Docker alternative for Google Colab)
print("📦 Installing udocker...")
!pip install udocker -q
!udocker --allow-root install > /dev/null 2>&1

# Pull the pre-configured Rxiv-Maker Docker image
print("📥 Pulling Rxiv-Maker container (henriqueslab/rxiv-maker-base:latest)...")
print("This container includes:")
print("  - Complete LaTeX distribution (TeX Live)")
print("  - Python 3.11 with scientific libraries")
print("  - R with common packages")
print("  - Node.js 18 with Mermaid CLI")
print("  - All system dependencies")

!udocker --allow-root pull henriqueslab/rxiv-maker-base:latest

# Create a container instance
print("\n🔧 Creating container instance...")
!udocker --allow-root create --name=rxiv-maker henriqueslab/rxiv-maker-base:latest

# Install ezinput for UI components
print("\n📱 Installing UI components...")
!pip install ezinput -q

print("\n✅ Docker environment ready!")
print("Container 'rxiv-maker' created with all dependencies pre-installed.")

## 💾 Choose between storing the files in current Colab Session or in your Google Drive

In [ ]:
#@title Where do you want to store the files?
import os
from ezinput import EZInput as ezi
from IPython.display import clear_output
from google.colab import output

def print_set(options):
  nLabels = gui._nLabels
  gui[f"label_{nLabels}"].value = "All settings saved!"

gui = ezi()
gui.add_label("Do you want to save the cloned repository and generated files in Google Drive?")
gui.add_label("If not, cloned repo and files will be stored in the current Colab runtime.")
gui.add_check("in_gdrive", description="Store in Google Drive?", remember_value=True, value=False)
gui.add_label("   ")
gui.add_label("Type the name for the manuscript folder to be used.")
gui.add_label("If a name different from the default MANUSCRIPT folder is selected for the first time, make sure to also select 'Create new manuscript folder'.")
gui.add_text_area("man_name", description="Manuscript Name:", remember_value=True, placeholder="MANUSCRIPT", value="MANUSCRIPT")
gui.add_label("   ")
gui.add_label("Select if you are using a new name for the manuscript folder.")
gui.add_check("new_folder", description="Create new manuscript folder", remember_value=True, value=False)
gui.add_label("   ")
gui.add_label("Select 'Continue a previous build' to use previously created files stored in your Google Drive.")
gui.add_label("If not selected a new default folder will be created and possibly overwrite previously created files.")
gui.add_check("continue", description="Continue a previous build?", remember_value=True, value=False)
gui.add_label("   ")
gui.add_callback("set", print_set, {}, description="Set options")
gui.add_label("   ")
gui.show()

## 📥 Clone Rxiv-Maker Repository

In [ ]:
#@title Clone the Rxiv-Maker repository to access the processing scripts and templates.
import os
import shutil

if gui["in_gdrive"].value:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/rxiv-maker"
    os.environ["MANUSCRIPT_PATH"] = "drive/MyDrive/rxiv-maker" + os.sep + gui["man_name"].value
else:
    PATH = "/content/rxiv-maker"
    os.environ["MANUSCRIPT_PATH"] = "rxiv-maker" + os.sep + gui["man_name"].value

output_path = PATH + os.sep + "output"
output_figures_path = PATH + os.sep + gui["man_name"].value + os.sep + "FIGURES"

if not gui["continue"].value:
  # Clone the repository
  if os.path.exists(PATH):
      shutil.rmtree(PATH)

  !git clone https://github.com/HenriquesLab/rxiv-maker.git {PATH}

  # Create output directory
  !mkdir -p {output_path}
  !mkdir -p {output_figures_path}

  print("✅ Rxiv-Maker repository cloned and set up!")
  print(f"Current directory: {os.getcwd()}")

  if gui["new_folder"].value:
    if os.path.exists(PATH + os.sep + gui["man_name"].value):
        shutil.rmtree(PATH + os.sep + gui["man_name"].value)
    shutil.copytree(PATH + os.sep + "MANUSCRIPT", PATH + os.sep + gui["man_name"].value)
  elif gui["man_name"].value == "EXAMPLE_MANUSCRIPT":
    pass # will use the example manuscript folder
  else:
    os.rename(PATH + os.sep + "MANUSCRIPT", PATH + os.sep + gui["man_name"].value)

# Install rxiv-maker in the container
print("\n📦 Installing Rxiv-Maker in container...")
!udocker --allow-root run -v /content:/workspace rxiv-maker bash -c "cd /workspace{PATH.replace('/content', '')} && pip install -e . -q"

print("\n✅ Repository setup completed!")

## 📄 Generate PDF Article

In [ ]:
#@title Run setup, generate figures, compile LaTeX and generate PDF
MANUSCRIPT_FOLDER = f"/workspace{PATH.replace('/content', '')}" + os.sep + gui["man_name"].value
!udocker --allow-root run -v /content:/workspace rxiv-maker bash -c "cd /workspace{PATH.replace('/content', '')} && make setup MANUSCRIPT_PATH={MANUSCRIPT_FOLDER} && make clean-figures MANUSCRIPT_PATH={MANUSCRIPT_FOLDER} && make pdf MANUSCRIPT_PATH={MANUSCRIPT_FOLDER}"

## 📱 Preview and Download

In [ ]:
#@title View the generated PDF and download all output files.
import base64
from IPython.display import HTML, display

# Robust PDF detection - try both possible names
primary_pdf_path = os.path.join(output_path, gui["man_name"].value + ".pdf")
fallback_pdf_path = os.path.join(output_path, "MANUSCRIPT.pdf")

if os.path.exists(primary_pdf_path):
    pdf_path = primary_pdf_path
    pdf_name = gui["man_name"].value + ".pdf"
    print(f"📄 PDF generated successfully using Docker: {pdf_name}")
elif os.path.exists(fallback_pdf_path):
    pdf_path = fallback_pdf_path
    pdf_name = "MANUSCRIPT.pdf"
    print(f"📄 PDF generated successfully using Docker: {pdf_name}")
    print("⚠️  Note: PDF was generated with default name instead of manuscript name")
else:
    pdf_path = None
    pdf_name = None
    print("❌ PDF not found. Please check the compilation step above.")
    print("\nAvailable files in output directory:")
    !ls -la {output_path}

if pdf_path:
    # Display PDF preview
    with open(pdf_path, "rb") as f:
        pdf_data = f.read()

    pdf_b64 = base64.b64encode(pdf_data).decode("utf-8")
    pdf_display = f'<iframe src="data:application/pdf;base64,{pdf_b64}" width="100%" height="600px"></iframe>'
    display(HTML(pdf_display))

    print("\n" + "=" * 50)
    print("📥 Download your files:")
    print("Run the next cells to download the manuscript files and/or all files as a ZIP.")
    print("=" * 50)

In [ ]:
#@title Download Manuscript Files
from google.colab import files

# Robust PDF detection - try both possible names
primary_pdf_path = os.path.join(output_path, gui["man_name"].value + ".pdf")
fallback_pdf_path = os.path.join(output_path, "MANUSCRIPT.pdf")

if os.path.exists(primary_pdf_path):
    pdf_path = primary_pdf_path
    pdf_name = gui["man_name"].value + ".pdf"
    tex_name = gui["man_name"].value + ".tex"
elif os.path.exists(fallback_pdf_path):
    pdf_path = fallback_pdf_path
    pdf_name = "MANUSCRIPT.pdf"
    tex_name = "MANUSCRIPT.tex"
else:
    pdf_path = None
    pdf_name = None
    tex_name = None

# Download individual files
print("Click to download individual files:")
print("-" * 40)

# Download PDF
if pdf_path:
    files.download(pdf_path)
    print(f"✅ {pdf_name} downloaded")
else:
    print("❌ PDF not found - check compilation step")

# Download LaTeX source
if tex_name:
    tex_path = os.path.join(output_path, tex_name)
    if os.path.exists(tex_path):
        files.download(tex_path)
        print(f"✅ {tex_name} downloaded")
    else:
        print(f"❌ {tex_name} not found")

print("\n📦 Or download everything as ZIP:")

In [ ]:
#@title Download all files as a ZIP
import datetime
import zipfile

# Create ZIP archive with all output files
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"rxiv_maker_docker_output_{timestamp}.zip"

with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    # Add all files from output directory
    for root, dirs, files_list in os.walk(output_path):
        for file in files_list:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, output_path)
            zipf.write(file_path, arcname)
            print(f"📁 Added to ZIP: {arcname}")

print(f"\n✅ Created ZIP archive: {zip_filename}")
print(f"📦 Archive size: {os.path.getsize(zip_filename)} bytes")

# Download the ZIP file
files.download(zip_filename)
print("🎉 ZIP archive downloaded successfully!")
print("\n🐳 Generated using Docker-accelerated Rxiv-Maker")

## 🔧 Troubleshooting

In [ ]:
#@title If you encounter issues, run this cell to check common problems.
print("🔍 TROUBLESHOOTING CHECKLIST (Docker Version)")
print("=" * 50)

# Check Docker container
print("🐳 Checking Docker container...")
!udocker --allow-root ps

# Check required files
print("\n📁 Checking required files...")
required_files = [
    PATH + os.sep + gui["man_name"].value + os.sep + "01_MAIN.md",
    PATH + os.sep + gui["man_name"].value + os.sep + "03_REFERENCES.bib"
]
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file} found")
    else:
        print(f"❌ {file} missing")

# Check Python scripts
print("\n🐍 Checking Python scripts...")
python_scripts = [
    PATH + os.sep + "src/py/commands/generate_preprint.py",
    PATH + os.sep + "src/py/commands/generate_figures.py",
]
for script in python_scripts:
    if os.path.exists(script):
        print(f"✅ {script} found")
    else:
        print(f"❌ {script} missing")

# Check LaTeX installation in container
print("\n📝 Checking LaTeX in Docker container...")
!udocker --allow-root run rxiv-maker pdflatex --version | head -1

# Check output directory
print("\n📂 Checking output directory...")
if os.path.exists(output_path):
    print("✅ Output directory exists")
    files_in_output = os.listdir(output_path)
    print(f"📁 Files in output: {len(files_in_output)}")

    # Robust PDF detection for troubleshooting
    primary_pdf_name = gui["man_name"].value + ".pdf"
    fallback_pdf_name = "MANUSCRIPT.pdf"
    primary_tex_name = gui["man_name"].value + ".tex"
    fallback_tex_name = "MANUSCRIPT.tex"
    primary_log_name = gui["man_name"].value + ".log"
    fallback_log_name = "MANUSCRIPT.log"

    # Check TEX files
    if primary_tex_name in files_in_output:
        print(f"✅ {primary_tex_name} generated")
    elif fallback_tex_name in files_in_output:
        print(f"✅ {fallback_tex_name} generated (fallback)")
    else:
        print(f"❌ No TEX file found ({primary_tex_name} or {fallback_tex_name})")

    # Check PDF files
    if primary_pdf_name in files_in_output:
        print(f"✅ {primary_pdf_name} generated")
    elif fallback_pdf_name in files_in_output:
        print(f"✅ {fallback_pdf_name} generated (fallback)")
    else:
        print(f"❌ No PDF file found ({primary_pdf_name} or {fallback_pdf_name})")
else:
    print("❌ Output directory missing")

print("\n" + "=" * 50)
print("If you see any ❌ errors above, please:")
print("1. Re-run the setup cells")
print("2. Check that your input files are properly formatted")
print("3. Look at the LaTeX log for compilation errors")
print("4. Verify the Docker container is running properly")

# Show recent LaTeX log if available
log_found = False
for log_name in [primary_log_name, fallback_log_name]:
    log_path = os.path.join(output_path, log_name)
    if os.path.exists(log_path):
        print(f"\n📄 Recent LaTeX log from {log_name} (last 20 lines):")
        !tail -20 {log_path}
        log_found = True
        break

if not log_found:
    print("\n📄 No LaTeX log files found")

## 📖 Next Steps

**Congratulations!** You've successfully used the Docker-accelerated Rxiv-Maker to generate a publication-ready LaTeX article.

### 🐳 Docker-Accelerated Benefits:

- **⚡ Faster setup**: ~2 minutes vs ~15 minutes for manual installation
- **🔒 Reliable**: Pre-configured environment with all dependencies
- **🔄 Consistent**: Identical results across different platforms
- **🛡️ Isolated**: No conflicts with Google Colab's environment

### What you can do next:

1. **Customize your manuscript**: Edit the YAML frontmatter in your Markdown file to change title, authors, affiliations, etc.

2. **Add more figures**: Create Python scripts or Mermaid diagrams in the FIGURES directory

3. **Modify the LaTeX template**: The template is in `src/tex/template.tex`

4. **Use locally with Docker**: Run `make pdf RXIV_ENGINE=DOCKER` on your local machine

5. **Switch to local installation**: Clone the repository and run `make setup && make pdf`

### Learn more:
- [Rxiv-Maker Documentation](https://github.com/HenriquesLab/rxiv-maker)
- [Docker Engine Mode Guide](https://github.com/HenriquesLab/rxiv-maker/blob/main/docs/workflows/docker-engine-mode.md)
- [Markdown Guide](https://www.markdownguide.org/)
- [LaTeX Documentation](https://www.latex-project.org/help/documentation/)

### Need help?
- Open an issue on [GitHub](https://github.com/HenriquesLab/rxiv-maker/issues)
- Check the troubleshooting section above
- For Docker-specific issues, see the [Docker Engine Mode Guide](https://github.com/HenriquesLab/rxiv-maker/blob/main/docs/workflows/docker-engine-mode.md)

### Technical Notes:
- This notebook uses `udocker` to run Docker containers in Google Colab
- The `henriqueslab/rxiv-maker-base:latest` image is optimized for AMD64 but works on all platforms
- Container includes: LaTeX, Python 3.11, R, Node.js, and all scientific libraries

---

*This Docker-accelerated notebook was created for the Rxiv-Maker project by the HenriquesLab.*